In [1]:
api_key= "ea0518bbe4e05ba4b9685668b348a1d7"

api_private_key = "6704343727a4d01b047aec1033b8478f540932c9"

url_base = "http://gateway.marvel.com"

endpoint_lists = "/v1/public/characters"


In [2]:
import requests

In [3]:
# Provide a list of 30 Marvel characters

url_list = url_base + endpoint_lists

import time

ts = 1000

ts_str = str(int(ts))

import hashlib

hash1 = str(ts_str)+api_private_key+api_key

hash2= hashlib.md5(hash1.encode())

hash3= hash2.hexdigest()


body = {'limit':30, 'ts':ts_str, 'apikey': api_key, 'hash':hash3}

characters = requests.get(url=url_list, params= body).json()

<b>1. Provide a list of 30 Marvel characters</b>

In [4]:
list_characters=[]
for i in characters['data']['results']:
    for j in i:
        if j == 'name':
            list_characters.append(i[j])

In [5]:
print(list_characters)

['3-D Man', 'A-Bomb (HAS)', 'A.I.M.', 'Aaron Stack', 'Abomination (Emil Blonsky)', 'Abomination (Ultimate)', 'Absorbing Man', 'Abyss', 'Abyss (Age of Apocalypse)', 'Adam Destine', 'Adam Warlock', 'Aegis (Trey Rollins)', 'Aero (Aero)', 'Agatha Harkness', 'Agent Brand', 'Agent X (Nijo)', 'Agent Zero', 'Agents of Atlas', 'Aginar', 'Air-Walker (Gabriel Lan)', 'Ajak', 'Ajaxis', 'Akemi', 'Alain', 'Albert Cleary', 'Albion', 'Alex Power', 'Alex Wilder', 'Alexa Mendez', 'Alexander Pierce']


<b>2. Retrieve the Ids for all the characters in your list (in str form)</b>

In [6]:
list_ids=[]
for i in characters['data']['results']:
    for j in i:
        if j == 'id':
            list_ids.append(str(i[j]))

In [7]:
print(list_ids)

['1011334', '1017100', '1009144', '1010699', '1009146', '1016823', '1009148', '1009149', '1010903', '1011266', '1010354', '1010846', '1017851', '1012717', '1011297', '1011031', '1009150', '1011198', '1011175', '1011136', '1011176', '1010870', '1011194', '1011170', '1009240', '1011120', '1010836', '1010755', '1011214', '1009497']


<b>3. Retrieve the total number of Events available for all the characters in your list (in integer form)</b>

In [35]:
endpoint_events = "/v1/public/characters/{characterId}/events".format(characterId=1011334)

In [46]:
body_events= { 'ts':ts_str,'apikey': api_key,'hash':hash3}
list_events = []
for i in list_ids:
    endpoint_events = "/v1/public/characters/{characterId}/events".format(characterId=int(i))
    url_final_events = url_base + endpoint_events
    c= requests.get(url=url_final_events, params= body_events).json()
    list_events.append(c['data']['count'])

In [47]:
list_events

[1,
 0,
 0,
 0,
 1,
 0,
 4,
 1,
 1,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

<b>4. Retrieve the total number of Series available for all the characters in your list  (in integer form)</b>

In [48]:
body_series= {'ts':ts_str,'apikey': api_key,'hash':hash3} #repeated from the previous
list_series = []
for i in list_ids:
    endpoint_events = "/v1/public/characters/{characterId}/series".format(characterId=int(i))
    url_final_series = url_base + endpoint_events
    c= requests.get(url=url_final_series, params= body_series).json()
    list_series.append(c['data']['count'])            

In [49]:
list_series

[3,
 2,
 20,
 3,
 20,
 2,
 20,
 3,
 3,
 0,
 20,
 0,
 5,
 10,
 7,
 3,
 10,
 13,
 0,
 3,
 1,
 0,
 0,
 0,
 0,
 1,
 6,
 4,
 0,
 1]

<b>5. Retrieve the total number of Comics available for all the characters in your list (in integer form)</b>

In [50]:
body_comics= {'ts':ts_str,'apikey': api_key,'hash':hash3} #repeated from the previous
list_comics = []
for i in list_ids:
    endpoint_comics = "/v1/public/characters/{characterId}/comics".format(characterId=int(i))
    url_final_comics = url_base + endpoint_comics
    c= requests.get(url=url_final_comics, params= body_comics).json()
    list_comics.append(c['data']['count']) 

In [51]:
list_comics

[12,
 4,
 20,
 14,
 20,
 2,
 20,
 8,
 3,
 0,
 20,
 0,
 20,
 20,
 20,
 18,
 20,
 20,
 0,
 4,
 4,
 0,
 0,
 0,
 0,
 1,
 18,
 9,
 0,
 1]

<b> 6. Retrieve the Price of the most expensive comic that the character was featured in or all the characters in your list (in float form and USD)</b>

In [86]:
#Needs to be completed
body_prices= {'ts':ts_str,'apikey': api_key,'hash':hash3} #repeated from the previous
list_prices = []
for i in list_ids:
    endpoint_comics = "/v1/public/characters/{characterId}/comics".format(characterId=int(i))
    url_final_comics = url_base + endpoint_comics
    c= requests.get(url=url_final_comics, params= body_comics).json()
    for k in c['data']['results']:
        list_prices_ind = 0
        for j in i['prices']:
            print (j)

TypeError: string indices must be integers

<b>7. Store the data above in a pandas DataFrame</b>

In [63]:
import pandas as pd

In [72]:
df = pd.DataFrame({'Character Name':list_characters, 'Character ID':list_ids,'Total Available Events':list_events,'Total Available Series':list_series,'Total Available Comics':list_comics })

In [73]:
df

,Character Name,Character ID,Total Available Events,Total Available Series,Total Available Comics
0,3-D Man,1011334,1,3,12
1,A-Bomb (HAS),1017100,0,2,4
2,A.I.M.,1009144,0,20,20
3,Aaron Stack,1010699,0,3,14
4,Abomination (Emil Blonsky),1009146,1,20,20
5,Abomination (Ultimate),1016823,0,2,2
6,Absorbing Man,1009148,4,20,20
7,Abyss,1009149,1,3,8
8,Abyss (Age of Apocalypse),1010903,1,3,3
9,Adam Destine,1011266,0,0,0
